#### Install Selenium & Import Necessary Packages

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#### Connect Selenium to Chrome and Direct to Desired URL

In [2]:
browser = webdriver.Chrome(executable_path  = '/Users/stephaniekendall/Library/Application Support/Google/chromedriver')

In [3]:
browser.get('https://www.waterqualitydata.us/portal/#countrycode=US&sampleMedia=Water&sampleMedia=water&startDateLo=01-01-2014&startDateHi=01-01-2019&mimeType=csv&dataProfile=biological')

#### Create a Function for Downloading State CSVs

In [4]:
def get_csv(x):
    #ensure all location parameters are clear
#     clear_field = browser.find_element_by_xpath('//*[@title="Remove all items"]')
#     clear_field.click()
    
# #     # input US as Country parameter
#     country = browser.find_element_by_xpath('//*[@class="select2-search__field"]')
#     country.send_keys('US', Keys.ENTER)
    
    # input x as State parameter
    element = browser.find_element_by_xpath("//input[@placeholder='All'][1]")
    element.send_keys(x, Keys.ENTER)
    
    # Download Biological metadata sample results
    results_button = browser.find_element_by_id("samples")
    results_button.click()
    browser.implicitly_wait(5)
    
    # submit and download results 
    download_button = browser.find_element_by_xpath('//*[@type="submit"]')
    download_button.click()
    
    # wait until query loads to press download
    browser.implicitly_wait(15)
    continue_button = browser.find_element_by_xpath('//*[@id="progress-continue-btn"]')
    continue_button.click()

In [6]:
get_csv('')

#### Unzip Files

In [59]:
from zipfile import ZipFile

def unzipfile(x):
    with ZipFile(x, 'r') as zip:
        zip.printdir()
        zip.extractall()

In [65]:
unzipfile('biologicalresult1.zip')

File Name                                             Modified             Size
biologicalresult.csv                           2019-11-12 16:39:30    176633921


#### Clean Data

In [9]:
import pandas as pd

In [10]:
USdf = pd.read_csv('result.csv')

/Users/stephaniekendall/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5,9,10,11,12,13,14,16,18,20,22,23,24,25,30,33,35,37,39,40,42,43,44,47,48,51,54,55,56,57,59,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
USdf = USdf[['MonitoringLocationIdentifier']]

In [12]:
identifiers = USdf.values.tolist()

In [13]:
len(identifiers)

417085

In [14]:
flat_list = []
for x in identifiers:
    for y in x:
        flat_list.append(y)

In [15]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [16]:
unique_ids = f7(flat_list)

In [17]:
govloc = []
siteloc = []
for ids in unique_ids:
    govloc.append(ids.split('-')[0])
    siteloc.append(ids.split('-')[-1])

In [18]:
def merge(list1,list2):
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

In [19]:
all_locs = merge(govloc, siteloc)
#list of tuples

In [20]:
all_locs[1]

('USGS', '15024800')

In [21]:
use_locs = dict(zip(govloc, siteloc))

In [22]:
urls = []
for locs in all_locs:
    url = 'https://waterdata.usgs.gov/nwis/nwismap/?site_no='+locs[1]+'&agency_cd='+locs[0]
    urls.append(url)

In [23]:
urls[:3]

['https://waterdata.usgs.gov/nwis/nwismap/?site_no=15015595&agency_cd=USGS',
 'https://waterdata.usgs.gov/nwis/nwismap/?site_no=15024800&agency_cd=USGS',
 'https://waterdata.usgs.gov/nwis/nwismap/?site_no=15041200&agency_cd=USGS']

In [24]:
len(urls)

27373

In [25]:
subs = 'USGS'
real_urls = [i for i in urls if subs in i]

In [26]:
len(real_urls)

7656

#### Get County Data for USGS codes

In [27]:
import requests
from bs4 import BeautifulSoup

In [30]:
import timeit

In [ ]:
counties_info = []
for url in real_urls:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
    table = soup.find('div',{'align':'left'})
    county_state = table.text.strip()
    print(county_state)
    counties_info.append(county_state)
   
    
#     counties_info.append(table)
    
#     
#     counties_info.append(county_state)
    
#     for row in table.find_all('tr'):
#         column_marker = 0            
#         columns = row.find_all('td')
#         all_counties.append([column.get_text() for column in columns])

Ketchikan Gateway Borough, Alaska
City And Borough Of Wrangell, Alaska
City And Borough Of Juneau, Alaska
Haines Borough, Alaska
Matanuska-Susitna Borough, Alaska
Matanuska-Susitna Borough, Alaska
Matanuska-Susitna Borough, Alaska
Matanuska-Susitna Borough, Alaska
Matanuska-Susitna Borough, Alaska
Matanuska-Susitna Borough, Alaska
Matanuska-Susitna Borough, Alaska
Lake And Peninsula Borough, Alaska
Houston County, Alabama
Escambia County, Alabama
Conecuh County, Alabama
Choctaw County, Alabama
Lee County, Alabama
Chilton County, Alabama
Talladega County, Alabama
Stone County, Arkansas
Union County, Arkansas
Union County, Arkansas
Union County, Arkansas
Chicot County, Arkansas
Union County, Arkansas
Ashley County, Arkansas
Chicot County, Arkansas
Ashley County, Arkansas
Union County, Arkansas
Chicot County, Arkansas
Chicot County, Arkansas
Calhoun County, Arkansas
Chicot County, Arkansas
Ouachita County, Arkansas
Ouachita County, Arkansas
Little River County, Arkansas
Hempstead County, 

Riverside County, California
Orange County, California
Riverside County, California
Los Angeles County, California
Los Angeles County, California
Orange County, California
Riverside County, California
Los Angeles County, California
Los Angeles County, California
Riverside County, California
Orange County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Riverside County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Orange County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Riverside County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Los Angeles County, California
Orange County, California
Riverside Coun

Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
San Bernardino County, California
Santa Barbara County, California
Los Angeles County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Santa Barbara County, California
Los Angeles County, California
Santa Barbara County, California
San Bernardino County, California
Santa Barbara County, California
Santa Barbara County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San

San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
Kern County, California
Kern County, California
San Bernardino County, California
Kern County, California
Kern County, California
Kern County, California
San Bernardino County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, California
San Bernardino County, California
Kern County, California
San Bernardino County, California
San Bernardino County, California
Kern County, California
Kern County, California
Kern County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
San Bernardino County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, California
Kern County, Califor

Santa Clara County, California
Madera County, California
Madera County, California
Santa Cruz County, California
Santa Cruz County, California
Santa Cruz County, California
Santa Cruz County, California
Madera County, California
Santa Cruz County, California
Santa Cruz County, California
Santa Cruz County, California
Inyo County, California
Merced County, California
Madera County, California
Madera County, California
Madera County, California
Merced County, California
Inyo County, California
Merced County, California
Santa Clara County, California
Santa Clara County, California
Merced County, California
Madera County, California
Madera County, California
Merced County, California
Santa Cruz County, California
Merced County, California
Madera County, California
Merced County, California
Santa Clara County, California
Merced County, California
Merced County, California
Merced County, California
Santa Clara County, California
Inyo County, California
Madera County, California
Merced County

...

Bradford County, Pennsylvania
Bradford County, Pennsylvania
Bradford County, Pennsylvania
Potter County, Pennsylvania
Barceloneta Municipio, Puerto Rico
Arecibo Municipio, Puerto Rico
San Sebastian Municipio, Puerto Rico
Quebradillas Municipio, Puerto Rico
Adjuntas Municipio, Puerto Rico
Utuado Municipio, Puerto Rico
Jayuya Municipio, Puerto Rico
Utuado Municipio, Puerto Rico
Utuado Municipio, Puerto Rico
Utuado Municipio, Puerto Rico
Arecibo Municipio, Puerto Rico
Corozal Municipio, Puerto Rico
Orocovis Municipio, Puerto Rico
Morovis Municipio, Puerto Rico
Ciales Municipio, Puerto Rico
Ciales Municipio, Puerto Rico
Ciales Municipio, Puerto Rico
Ciales Municipio, Puerto Rico
Manati Municipio, Puerto Rico
Manati Municipio, Puerto Rico
Vega Baja Municipio, Puerto Rico
Vega Baja Municipio, Puerto Rico
Cayey Municipio, Puerto Rico
Cayey Municipio, Puerto Rico
Aibonito Municipio, Puerto Rico
Comerio Municipio, Puerto Rico
Comerio Municipio, Puerto Rico
Naranjito Municipio, Puerto Rico
Dorad

Williamson County, Tennessee
Williamson County, Tennessee
Williamson County, Tennessee
Williamson County, Tennessee
Williamson County, Tennessee
Williamson County, Tennessee
Gibson County, Tennessee
Unicoi County, Tennessee
Unicoi County, Tennessee
Unicoi County, Tennessee
Dyer County, Tennessee
Dyer County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Fentress County, Tennessee
Tyler County, Texas
Navarro County, Texas
Freestone County, Texas
Liberty County, Texas
Harris County, Texas
Montgomery County, Texas
Harris County, Texas
Montgomery County, Texas
Harris County, Texas
Harris County, Texas
Harris County, Texas
Harris County, Texas
Harris County, Texas
Harris County, Texas
Harris County, Texas
Harris County, Texas
Travis County, Texas
Travis County,

Panola County, Texas
Johnson County, Texas
Rusk County, Texas
Smith County, Texas
Harrison County, Texas
Ellis County, Texas
Gaines County, Texas
Smith County, Texas
Gregg County, Texas
Upshur County, Texas
Gaines County, Texas
Gaines County, Texas
Gaines County, Texas
Gaines County, Texas
Dallas County, Texas
Gaines County, Texas
Dawson County, Texas
Gaines County, Texas
Gaines County, Texas
Gaines County, Texas
Gaines County, Texas
Tarrant County, Texas
Gaines County, Texas
Gaines County, Texas
Yoakum County, Texas
Yoakum County, Texas
Yoakum County, Texas
Yoakum County, Texas
Camp County, Texas
Terry County, Texas
Cass County, Texas
Terry County, Texas
Yoakum County, Texas
Yoakum County, Texas
Yoakum County, Texas
Yoakum County, Texas
Terry County, Texas
Terry County, Texas
Yoakum County, Texas
Terry County, Texas
Terry County, Texas
Yoakum County, Texas
Terry County, Texas
Yoakum County, Texas
Terry County, Texas
Terry County, Texas
Terry County, Texas
Terry County, Texas
Yoakum Co

Kitsap County, Washington
Spokane County, Washington
Lincoln County, Washington
King County, Washington
King County, Washington
Grant County, Washington
Kitsap County, Washington
Douglas County, Washington
King County, Washington
Spokane County, Washington
Lincoln County, Washington
Lincoln County, Washington
Kitsap County, Washington
Snohomish County, Washington
Grant County, Washington
Grant County, Washington
Snohomish County, Washington
Douglas County, Washington
Snohomish County, Washington
Snohomish County, Washington
Island County, Washington
Ferry County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County, Washington
Whatcom County

Ritchie County, West Virginia
Ritchie County, West Virginia
Doddridge County, West Virginia
Berkeley County, West Virginia
Doddridge County, West Virginia
Tyler County, West Virginia
Tyler County, West Virginia
Mineral County, West Virginia
Wetzel County, West Virginia
Tyler County, West Virginia
Wetzel County, West Virginia
Wetzel County, West Virginia
Wetzel County, West Virginia
Monongalia County, West Virginia
Wetzel County, West Virginia
Wetzel County, West Virginia
Brooke County, West Virginia
Hancock County, West Virginia


In [38]:
union_county = 'https://waterdata.usgs.gov/nwis/nwismap/?site_no=330218092112401&agency_cd=USGS'

In [44]:
responses = requests.get(union_county)
soups = BeautifulSoup(responses.content, 'html.parser')
tables = soups.find('div',{'align':'left'})
# name = tables.text.strip()

In [59]:
len(counties_info)

7655

#### Create DataFrame from ID and Counties

In [60]:
countyid = dict(zip(unique_ids, counties_info))

In [61]:
countyid

{'USGS-15015595': 'Ketchikan Gateway Borough, Alaska',
 'USGS-15024800': 'City And Borough Of Wrangell, Alaska',
 'USGS-15041200': 'City And Borough Of Juneau, Alaska',
 'USGS-15056500': 'Haines Borough, Alaska',
 'USGS-15291700': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15292000': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15292100': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15292400': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15292780': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15294345': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15294350': 'Matanuska-Susitna Borough, Alaska',
 'USGS-15298040': 'Lake And Peninsula Borough, Alaska',
 'USGS-310153085225901': 'Houston County, Alabama',
 'USGS-310338087101502': 'Escambia County, Alabama',
 'USGS-312510087142601': 'Conecuh County, Alabama',
 'USGS-315621088271901': 'Choctaw County, Alabama',
 'USGS-323500085274801': 'Lee County, Alabama',
 'USGS-325740086432401': 'Chilton County, Alabama',
 'USGS-330636086251101': 'Talladega C

In [64]:
countyids = pd.DataFrame.from_dict(countyid, orient='index')

In [70]:
countyids.columns = ['County']
countyids.head()

,County
USGS-15015595,"Ketchikan Gateway Borough, Alaska"
USGS-15024800,"City And Borough Of Wrangell, Alaska"
USGS-15041200,"City And Borough Of Juneau, Alaska"
USGS-15056500,"Haines Borough, Alaska"
USGS-15291700,"Matanuska-Susitna Borough, Alaska"


In [75]:
counties_ids = pd.concat([countyids['County'], 
                           countyids.County.str.split(', ', expand=True)], axis=1)

In [76]:
counties_ids.head()

,County,0,1
USGS-15015595,"Ketchikan Gateway Borough, Alaska",Ketchikan Gateway Borough,Alaska
USGS-15024800,"City And Borough Of Wrangell, Alaska",City And Borough Of Wrangell,Alaska
USGS-15041200,"City And Borough Of Juneau, Alaska",City And Borough Of Juneau,Alaska
USGS-15056500,"Haines Borough, Alaska",Haines Borough,Alaska
USGS-15291700,"Matanuska-Susitna Borough, Alaska",Matanuska-Susitna Borough,Alaska


In [78]:
del counties_ids['County']

In [119]:
countiesdf = counties_ids
countiesdf.head()

,County,State
USGS-15015595,Ketchikan Gateway Borough,Alaska
USGS-15024800,City And Borough Of Wrangell,Alaska
USGS-15041200,City And Borough Of Juneau,Alaska
USGS-15056500,Haines Borough,Alaska
USGS-15291700,Matanuska-Susitna Borough,Alaska


In [89]:
len(countiesdf.State.unique())

51

In [133]:
countiesdf = countiesdf[countiesdf.State != 'Hawaii']
countiesdf = countiesdf[countiesdf.State != 'Puerto Rico']
countiesdf = countiesdf[countiesdf.State != 'Alaska']
len(countiesdf.State.unique())

49

In [121]:
countiesdf = countiesdf.reset_index()

In [134]:
countiesdf.columns = ('id', 'Counties', 'State')
countiesdf.head()

,id,Counties,State
12,USGS-310153085225901,Houston County,Alabama
13,USGS-310338087101502,Escambia County,Alabama
14,USGS-312510087142601,Conecuh County,Alabama
15,USGS-315621088271901,Choctaw County,Alabama
16,USGS-323500085274801,Lee County,Alabama


In [139]:
countiesdf.State.value_counts()

California           1374
Pennsylvania          684
Texas                 434
New York              426
New Jersey            390
Colorado              295
New Mexico            282
Idaho                 255
Missouri              223
North Dakota          187
Wyoming               169
Wisconsin             168
Washington            153
Michigan              151
Utah                  147
Kansas                146
Georgia               143
Arizona               136
West Virginia         124
Arkansas              117
Minnesota             106
Nebraska              105
Maryland              104
Illinois               97
Tennessee              95
Montana                91
South Carolina         90
Nevada                 89
Florida                88
Iowa                   81
Virginia               70
Oregon                 68
North Carolina         64
Connecticut            61
Massachusetts          54
Oklahoma               53
Louisiana              49
Mississippi            48
South Dakota

In [118]:
countiesdf.head()

0      Ketchikan Gateway Borough
1    ity And Borough Of Wrangell
2       ity And Borough Of Junea
3                 Haines Borough
4      Matanuska-Susitna Borough
Name: County, dtype: object

#### Connect to MYSQL and Insert Data

In [1]:
import mysql.connector
import config
import pandas as pd

In [2]:
#set up mysql connection
cnx = mysql.connector.connect(
   host = config.host,
   user = config.user,
   password = config.password)
cursor = cnx.cursor()

In [3]:
#specify database for connection
cnx = mysql.connector.connect(
   host = config.host,
   user = config.user,
   password = config.password,
   database = "water")
cursor = cnx.cursor()

In [4]:
create_query = """
     CREATE TABLE counties
   ( id VARCHAR(50) PRIMARY KEY,
     County VARCHAR(50),
     State VARCHAR(20)
     )
   """

In [113]:
cursor.execute(create_query)

In [114]:
subset = countiesdf[['id', 'County', 'State']]
tuples = [tuple(x) for x in subset.values]

In [115]:
def insert_datas(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO counties (id,
                                               County,
                                               State)
                           VALUES (%s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [116]:
insert_datas(countiesdf)

In [7]:
# cursor = cnx.cursor(buffered=True)
cursor.execute("""SELECT *
                      FROM water.counties;""")
counties = pd.DataFrame(cursor.fetchall())
counties.columns = [x[0] for x in cursor.description]
counties

ValueError: Length mismatch: Expected axis has 0 elements, new values have 3 elements

In [16]:
statesabv = {
        'AK': 'O','AL': 'S','AR': 'S','AS': 'O','AZ': 'W','CA': 'W','CO': 'W',
        'CT': 'N','DC': 'N','DE': 'N','FL': 'S','GA': 'S','GU': 'O','HI': 'O',
        'IA': 'M','ID': 'W','IL': 'M','IN': 'M','KS': 'M','KY': 'S','LA': 'S',
        'MA': 'N','MD': 'N','ME': 'N','MI': 'W','MN': 'M','MO': 'M','MP': 'O',
        'MS': 'S','MT': 'W','NA': 'O','NC': 'S','ND': 'M','NE': 'W','NH': 'N',
        'NJ': 'N','NM': 'W','NV': 'W','NY': 'N','OH': 'M','OK': 'S','OR': 'W',
        'PA': 'N','PR': 'O','RI': 'N','SC': 'S','SD': 'M','TN': 'S','TX': 'S',
        'UT': 'W','VA': 'S','VI': 'O','VT': 'N','WA': 'W','WI': 'M','WV': 'S',
        'WY': 'W'
        }

In [17]:
statesab2 = {
        'AK': 'Alaska','AL': 'Alabama', 'AR': 'Arkansas', 'AS': 'American Samoa',
        'AZ': 'Arizona','CA': 'California','CO': 'Colorado','CT': 'Connecticut',
        'DC': 'District of Columbia','DE': 'Delaware','FL': 'Florida','GA': 'Georgia',
        'GU': 'Guam','HI': 'Hawaii','IA': 'Iowa','ID': 'Idaho','IL': 'Illinois',
        'IN': 'Indiana','KS': 'Kansas','KY': 'Kentucky','LA': 'Louisiana',
        'MA': 'Massachusetts','MD': 'Maryland','ME': 'Maine','MI': 'Michigan',
        'MN': 'Minnesota','MO': 'Missouri','MP': 'Northern Mariana Islands',
        'MS': 'Mississippi','MT': 'Montana','NA': 'National','NC': 'North Carolina',
        'ND': 'North Dakota','NE': 'Nebraska','NH': 'New Hampshire','NJ': 'New Jersey',
        'NM': 'New Mexico','NV': 'Nevada','NY': 'New York','OH': 'Ohio',
        'OK': 'Oklahoma','OR': 'Oregon','PA': 'Pennsylvania','PR': 'Puerto Rico',
        'RI': 'Rhode Island','SC': 'South Carolina','SD': 'South Dakota',
        'TN': 'Tennessee','TX': 'Texas','UT': 'Utah','VA': 'Virginia',
        'VI': 'Virgin Islands','VT': 'Vermont','WA': 'Washington',
        'WI': 'Wisconsin','WV': 'West Virginia','WY': 'Wyoming'
}

In [18]:
state_regions = [statesab2, statesabv]
d = {}
for k in statesab2.keys():
    d[k] = tuple(d[k] for d in state_regions)

In [19]:
state_regions

[{'AK': 'Alaska',
  'AL': 'Alabama',
  'AR': 'Arkansas',
  'AS': 'American Samoa',
  'AZ': 'Arizona',
  'CA': 'California',
  'CO': 'Colorado',
  'CT': 'Connecticut',
  'DC': 'District of Columbia',
  'DE': 'Delaware',
  'FL': 'Florida',
  'GA': 'Georgia',
  'GU': 'Guam',
  'HI': 'Hawaii',
  'IA': 'Iowa',
  'ID': 'Idaho',
  'IL': 'Illinois',
  'IN': 'Indiana',
  'KS': 'Kansas',
  'KY': 'Kentucky',
  'LA': 'Louisiana',
  'MA': 'Massachusetts',
  'MD': 'Maryland',
  'ME': 'Maine',
  'MI': 'Michigan',
  'MN': 'Minnesota',
  'MO': 'Missouri',
  'MP': 'Northern Mariana Islands',
  'MS': 'Mississippi',
  'MT': 'Montana',
  'NA': 'National',
  'NC': 'North Carolina',
  'ND': 'North Dakota',
  'NE': 'Nebraska',
  'NH': 'New Hampshire',
  'NJ': 'New Jersey',
  'NM': 'New Mexico',
  'NV': 'Nevada',
  'NY': 'New York',
  'OH': 'Ohio',
  'OK': 'Oklahoma',
  'OR': 'Oregon',
  'PA': 'Pennsylvania',
  'PR': 'Puerto Rico',
  'RI': 'Rhode Island',
  'SC': 'South Carolina',
  'SD': 'South Dakota',
  'TN

In [21]:
region_list = list(d.values())
region_list

[('Alaska', 'O'),
 ('Alabama', 'S'),
 ('Arkansas', 'S'),
 ('American Samoa', 'O'),
 ('Arizona', 'W'),
 ('California', 'W'),
 ('Colorado', 'W'),
 ('Connecticut', 'N'),
 ('District of Columbia', 'N'),
 ('Delaware', 'N'),
 ('Florida', 'S'),
 ('Georgia', 'S'),
 ('Guam', 'O'),
 ('Hawaii', 'O'),
 ('Iowa', 'M'),
 ('Idaho', 'W'),
 ('Illinois', 'M'),
 ('Indiana', 'M'),
 ('Kansas', 'M'),
 ('Kentucky', 'S'),
 ('Louisiana', 'S'),
 ('Massachusetts', 'N'),
 ('Maryland', 'N'),
 ('Maine', 'N'),
 ('Michigan', 'W'),
 ('Minnesota', 'M'),
 ('Missouri', 'M'),
 ('Northern Mariana Islands', 'O'),
 ('Mississippi', 'S'),
 ('Montana', 'W'),
 ('National', 'O'),
 ('North Carolina', 'S'),
 ('North Dakota', 'M'),
 ('Nebraska', 'W'),
 ('New Hampshire', 'N'),
 ('New Jersey', 'N'),
 ('New Mexico', 'W'),
 ('Nevada', 'W'),
 ('New York', 'N'),
 ('Ohio', 'M'),
 ('Oklahoma', 'S'),
 ('Oregon', 'W'),
 ('Pennsylvania', 'N'),
 ('Puerto Rico', 'O'),
 ('Rhode Island', 'N'),
 ('South Carolina', 'S'),
 ('South Dakota', 'M'),
 ('Te

In [22]:
regiondict = dict(region_list)

In [23]:
counties['Region'] = counties['State'].map(regiondict)

In [24]:
counties.head()

,id,County,State,Region
0,AR001-343014091325401,Stone County,Arkansas,S
1,AR001-354459090334000,Union County,Arkansas,S
2,AR008-331818091233401,Union County,Arkansas,S
3,AR008-334144091284201,Union County,Arkansas,S
4,AR008-340740091211501,Chicot County,Arkansas,S


In [6]:
subset3 = counties[['id','State', 'County', 'Region']]
tuples3 = [tuple(x) for x in subset3.values]

KeyError: "None of [Index(['id', 'State', 'County', 'Region'], dtype='object')] are in the [columns]"

In [31]:
def insert_datas(dataframe):
    tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO counties (id,
                                               State,
                                               County,
                                               Region)
                           VALUES (%s, %s, %s, %s);"""
    cursor.executemany(insert_statement, tuples)
    cnx.commit()

In [ ]:
insert_datas(counties)

#### Add Median Income to Counties

In [1]:
def clean_csvs(x):
    x = pd.read_csv(x)
    x.rename(columns={'ActivityLocation/LatitudeMeasure': 'ActivityLatitude',
                       'ActivityLocation/LongitudeMeasure': 'ActivityLongitude',
                       'CharacteristicName':'Contaminant',
                       'ResultMeasureValue':'DetectedLevel',
                       'ResultMeasure/MeasureUnitCode':'LevelUnit'}, inplace=True)
    rows = ['Lead','Copper']
    x = x[x['Contaminant'].isin(rows)]
    x = x.dropna()
#     x = x.replace(to_replace = 'ND'

In [ ]:
clean_csvs('')